In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score, precision_score, recall_score

%load_ext tensorboard

In [2]:
InputData = np.loadtxt("dataInputLarge.csv",delimiter=",")
OutputData = np.loadtxt ("dataOutputLarge.csv", delimiter=",")

In [3]:
InputDataTrain = InputData[0:9000]  
InputDataTest = InputData[9000:11006]  

OutputDataTrain = OutputData[0:9000] 
OutputDataTest = OutputData[9000:11006]

In [4]:
scaler = MinMaxScaler(feature_range=(0, 1))

InputDataTrain_transformed = scaler.fit_transform(InputDataTrain) 
InputDataTrain_retransformed = scaler.inverse_transform(InputDataTrain_transformed)

InputDataTest_transformed = scaler.fit_transform(InputDataTest)
InputDataTest_retransformed = scaler.inverse_transform(InputDataTest_transformed)

OutputDataTrain_transformed = scaler.fit_transform(OutputDataTrain)
OutputDataTrain_retransformed = scaler.inverse_transform(OutputDataTrain_transformed)

OutputDataTest_transformed = scaler.fit_transform(OutputDataTest)
OutputDataTest_retransformed = scaler.inverse_transform(OutputDataTest_transformed)               


In [37]:
print(InputDataTest[1900])
print(InputDataTest_retransformed[1900])
print(InputDataTest-OutputDataTest_retransformed)

#https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/


[-12. -12. -12. -12.  12. -12.  12.  12. -12.  12.  12.  12.  12. -12.
 -12.  12. -12.  12. -12. -12.  12. -12.  12.  12. -12.  12.  12. -12.
 -12.  12.  12.]
[-12. -12. -12. -12.  12. -12.  12.  12. -12.  12.  12.  12.  12. -12.
 -12.  12. -12.  12. -12. -12.  12. -12.  12.  12. -12.  12.  12. -12.
 -12.  12.  12.]
[[ 0.55452327  4.67612859 -3.81396256 ...  2.86570075 -4.68610959
   1.03701825]
 [-4.96964927  8.96724901 -4.56483927 ... -7.42970781  0.71407962
  -0.68574012]
 [ 1.96517673  0.44657053  2.07828296 ... -6.288067    5.44359404
  -1.30383862]
 ...
 [-3.3349308  -0.91278207 -4.46239095 ...  1.6244886  -3.48224136
   1.48385858]
 [-4.94566445  4.15399795 -0.28108997 ...  6.64286411 -7.76491511
   0.68327844]
 [-2.10177883 -0.02345954 -1.16648202 ...  2.23600078 -0.60693802
  -0.74707724]]


In [6]:
def create_model() :
    return tf.keras.models.Sequential([
      tf.keras.Input(shape=((31,))),
      tf.keras.layers.Dense(62,input_dim = 31, activation='tanh'),
      tf.keras.layers.Dense(62,input_dim = 62, activation='tanh'),
      tf.keras.layers.Dense(31, input_dim = 62)
    ])

In [7]:
model = create_model()
model

In [8]:
model.summary()
model.output_shape

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 62)                1984      
_________________________________________________________________
dense_1 (Dense)              (None, 62)                3906      
_________________________________________________________________
dense_2 (Dense)              (None, 31)                1953      
Total params: 7,843
Trainable params: 7,843
Non-trainable params: 0
_________________________________________________________________


(None, 31)

In [9]:
scorers = {
        'precision_score': make_scorer(precision_score),
        'recall_score': make_scorer(recall_score),
        'accuracy_score': make_scorer(accuracy_score)
        }
param_grid = dict(epochs=[10,50,100])
Kmodel = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, verbose=1)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3,scoring=scorers,refit="precision_score")
grid_result = grid.fit(InputDataTrain_transformed, OutputDataTrain_transformed)

TypeError: Cannot clone object '<tensorflow.python.keras.engine.sequential.Sequential object at 0x000001C775423D00>' (type <class 'tensorflow.python.keras.engine.sequential.Sequential'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' method.

In [10]:
model.layers

In [20]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=InputDataTrain_transformed, 
          y=OutputDataTrain_transformed, 
          epochs=1000, 
          batch_size = 200,
          validation_data=(InputDataTest_transformed, OutputDataTest_transformed), 
          #callbacks=[tensorboard_callback]
         )

Epoch 1/1000

45/45 [==============================] - 0s 6ms/step - loss: 2.2822e-04 - accuracy: 0.8826 - val_loss: 1.0346e-04 - val_accuracy: 0.8295
Epoch 2/1000

45/45 [==============================] - 0s 2ms/step - loss: 4.1843e-05 - accuracy: 0.9332 - val_loss: 8.8882e-05 - val_accuracy: 0.8430
Epoch 3/1000

45/45 [==============================] - 0s 2ms/step - loss: 3.9678e-05 - accuracy: 0.9348 - val_loss: 8.8589e-05 - val_accuracy: 0.8405
Epoch 4/1000

45/45 [==============================] - 0s 2ms/step - loss: 4.0030e-05 - accuracy: 0.9346 - val_loss: 8.7965e-05 - val_accuracy: 0.8480
Epoch 5/1000

45/45 [==============================] - 0s 2ms/step - loss: 4.0911e-05 - accuracy: 0.9317 - val_loss: 8.8791e-05 - val_accuracy: 0.8435
Epoch 6/1000

45/45 [==============================] - 0s 2ms/step - loss: 4.1445e-05 - accuracy: 0.9317 - val_loss: 8.9823e-05 - val_accuracy: 0.8390
Epoch 7/1000

45/45 [==============================] - 0s 2ms/step - loss: 4.1287e-05 - accura

In [19]:
results = model.evaluate(InputDataTest_transformed,OutputDataTest_transformed)
print("Test loss, Test acc:", results)


 1/63 [..............................] - ETA: 0s - loss: 3.8545e-05 - accuracy: 1.0000WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0000s vs `on_test_batch_end` time: 0.0010s). Check your callbacks.

63/63 [==============================] - 0s 682us/step - loss: 8.9246e-05 - accuracy: 0.8390
Test loss, Test acc: [8.924554276745766e-05, 0.8389830589294434]


In [21]:
results = model.evaluate(InputDataTest_transformed,OutputDataTest_transformed)
print("Test loss, Test acc:", results)


63/63 [==============================] - 0s 571us/step - loss: 8.5600e-05 - accuracy: 0.8315
Test loss, Test acc: [8.560016431147233e-05, 0.8315054774284363]


In [13]:
# %tensorboard --logdir logs --host localhost --port 6003

In [41]:
first = 1905
second = 1906
predictions = model.predict(InputDataTest_transformed[first:second])
print("predictions shape:", predictions.shape)
print("InputDataTest:",InputDataTest_transformed[first:second])
print("InputDataTest in db (Command Gains)", InputDataTest_retransformed[first:second])
print("OutputDataTest:",OutputDataTest_transformed[first:second])
print("Predictions:",predictions)

print("OutputDataTest in dB:",scaler.inverse_transform(OutputDataTest_transformed[first:second]))
print("predictions in dB",scaler.inverse_transform(predictions))

predictions shape: (1, 31)
InputDataTest: [[1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1. 1. 1.
  0. 0. 1. 1. 1. 0. 1.]]
InputDataTest in db (Command Gains) [[ 12. -12.  12. -12. -12. -12.  12. -12. -12.  12. -12.  12.  12.  12.
  -12. -12.  12.  12.  12. -12.  12.  12.  12.  12. -12. -12.  12.  12.
   12. -12.  12.]]
OutputDataTest: [[0.95095382 0.01251795 0.98047762 0.22949495 0.35628845 0.21385772
  0.99171132 0.20588644 0.21285715 0.97505744 0.01116978 0.76683299
  0.62136876 0.79323524 0.20774307 0.20740807 0.80161067 0.62968791
  0.78229001 0.00237894 0.77977533 0.6240526  0.61200655 0.80095804
  0.20217355 0.20584085 0.79004794 0.63048946 0.79681798 0.01779445
  0.99966199]]
Predictions: [[ 0.9423985   0.03532252  0.98649997  0.2174975   0.37295195  0.21767682
   0.9728844   0.2218682   0.20648162  0.9637962   0.03201628  0.7835586
   0.6007454   0.7834566   0.2175899   0.21095446  0.7766522   0.6299487
   0.78248394 -0.00232916  0.7724631   0.62631416  0.605

In [42]:
model.output_shape

(None, 31)

In [43]:
%run Code.ipynb

C:/Users/Robert/AppData/Local/Temp/xpython_3932/2936004681.py:57: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  Gopt_db = np.linalg.lstsq(leak.conj().T, G_db2)[0]
C:/Users/Robert/AppData/Local/Temp/xpython_3932/4126492185.py:34: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  num = np.array([(1+G*beta), -2*cos(w0), (1-G*beta)]/(1+beta))
C:/Users/Robert/AppData/Local/Temp/xpython_3932/4126492185.py:35: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with 

In [44]:
commandGains = InputDataTest_retransformed[first:second].reshape((31,1)) 
#print(commandGains)
filterGainsPredicted = scaler.inverse_transform(predictions).reshape((31,1)) 
#print(filterGainsPredicted)

In [45]:
thirdOctaveGEQwithPredictions(commandGains,filterGainsPredicted)

C:/Users/Robert/AppData/Local/Temp/xpython_3932/2936004681.py:57: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  Gopt_db = np.linalg.lstsq(leak.conj().T, G_db2)[0]
C:/Users/Robert/AppData/Local/Temp/xpython_3932/4126492185.py:34: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  num = np.array([(1+G*beta), -2*cos(w0), (1-G*beta)]/(1+beta))
C:/Users/Robert/AppData/Local/Temp/xpython_3932/4126492185.py:35: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with 